---
#### Bouncing Ball Animation from Leaning Scientific Programming with Python

* Direct link to the [work](https://scipython.com/book2/chapter-7-matplotlib/examples/animating-a-bouncing-ball/)
---

---

##### Design

* A function to generate bouncing ball height based on co-efficient of restitution

* Generate a pandas dataframe to store for 4 different values of restitution. Dataframe should have time, height and co-efficient of restitution. We can regnerate all ODE values like reaction kinetics, but keeping the use case simple here.

* User button option for restitution

* Time slider for bouncing ball

* Animate button 

* Add Tool tips to have height of ball and time

---

In [21]:
import numpy as np
from bokeh.models import ColumnDataSource, Slider, Div, HoverTool, Grid, Tabs, Button
from bokeh.plotting import figure,show
from bokeh.io import output_notebook, curdoc
from bokeh.layouts import row, column, gridplot

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# Acceleration due to gravity, m.s-2.
g = 9.81
# The maximum x-range of ball's trajectory to plot.
XMAX = 5
# The coefficient of restitution for bounces (-v_up/v_down).
cor_start = 0.65
# The time step for the animation.
dt = 0.005
# Initial position and velocity vectors.
x0, y0, t0 = 0.0, 4.0, 0.0
vx0, vy0 = 1.0, 0.0

In [4]:
a=0
a+=1
a

1

In [5]:
initial_data = dict(x=[], y=[], t=[])
x, y, vx, vy, t = x0, y0, vx0, vy0, t0
print(y)
while x < XMAX:
    x += vx0 * dt
    y += vy * dt
    vy -= g * dt
    t += dt
    if y < 0:
        # bounce!
        y = 0
        vy = -vy * cor_start
    initial_data['x'].append(x)
    initial_data['y'].append(y)
    initial_data['t'].append(t)
source=ColumnDataSource(data=initial_data)
initial_pos_data = dict(x=[initial_data['x'][10]], y=[initial_data['y'][10]])
source_time = ColumnDataSource(data=initial_pos_data)

4.0


In [6]:
# Set up plot for concentrations
TOOLTIPS = [("x (m)","@x{0,0.000}"), ("y (m)","@y{0,0.000}")]
TOOLS = "pan,undo,redo,reset,save,wheel_zoom,box_zoom"
plot_ball = figure(height=450, width=550, tools=TOOLS, tooltips=TOOLTIPS,
              title="Bouncing Ball Animation")
plot_ball.line('x', 'y', source=source, line_width=3, line_alpha=0.6, line_color="mediumblue",
               legend_label="Trace")
plot_ball.circle('x', 'y', source=source_time, color="navy", size=15.0, alpha=0.75,
               legend_label="Current Location")
plot_ball.xaxis.axis_label = "x (m)"
plot_ball.yaxis.axis_label = "y (m)"
plot_ball.legend.location = "top_right"
plot_ball.legend.click_policy="hide"
plot_ball.legend.background_fill_alpha = 0.5
plot_ball.grid.grid_line_color = "silver"

In [7]:
show(plot_ball)

In [23]:
slider_cor = Slider(title="Co-efficient of Restitution"+" (initial: "+str(cor_start)+")", value=cor_start, start=0.1, end=1.0, step=0.05)

In [9]:
start_time = 0.0
end_time = 8.0
time_step = 0.1
slider_time = Slider(title="Time Slider (s)", value=start_time, start=start_time, end=end_time, step=time_step, width=500)

In [11]:
# Function to update the plot data
def update_data(attrname, old, new):
    cor = slider_cor.value
    time_value = slider_time.value
    new_data = dict(x=[], y=[], t=[])
    x, y, vx, vy, t = x0, y0, vx0, vy0, t0
    while x < XMAX:
        t += dt
        x += vx0 * dt
        y += vy * dt
        vy -= g * dt
        if y < 0:
            # bounce!
            y = 0
            vy = -vy * cor
        new_data['x'].append(x)
        new_data['y'].append(y)
        new_data['t'].append(t)
        index_time_value = new_data['t'].index(time_value)
        circle_dict = dict(x=[new_data['x'][index_time_value]], y=[new_data['y'][index_time_value]])
        source.data=new_data
        source_time.data=circle_dict

In [12]:
for w in [slider_cor, slider_time]:
    w.on_change('value', update_data)

In [13]:
def animate():
    global callback_id
    if animate_button.label == '► Play':
        animate_button.label = '❚❚ Pause'
        callback_id = curdoc().add_periodic_callback(animate_update, time_step*1000.0) # s to milliseconds conversion
    else:
        animate_button.label = '► Play'
        curdoc().remove_periodic_callback(callback_id)

animate_button = Button(label='► Play', width=50)
animate_button.on_event('button_click', animate)

In [19]:
# Set up layouts and add to document
layout_bouncing_ball = row(column(slider_cor, animate_button, slider_time), plot_ball)


In [22]:
# Setup server
curdoc().add_root(layout_bouncing_ball)

curdoc().title = "Bouncing Ball"
